In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from lazypredict.Supervised import LazyRegressor


In [2]:
df = pd.read_csv(r"C:\Users\DHONI HANIF\OneDrive\Documents\AI Collection and Loss Reverse Forecast\modelling\kolektor\regresi_for_time\data\kolektor.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             1000 non-null   int64  
 1   time_to_collect        1000 non-null   int64  
 2   avg_bill_methods       1000 non-null   object 
 3   debtor_volume_handled  1000 non-null   int64  
 4   bill_amount_collected  1000 non-null   int64  
 5   total_actual           1000 non-null   int64  
 6   total_cost             1000 non-null   int64  
 7   success_rate           1000 non-null   float64
 8   negotiation_skills     1000 non-null   object 
 9   collector_experience   1000 non-null   int64  
dtypes: float64(1), int64(7), object(2)
memory usage: 78.2+ KB


In [3]:
for i in df.columns:
    if df[i].dtype == "object":
        df[i] = df[i].astype("category")

df.drop(["Unnamed: 0"], axis=1, inplace=True)

In [4]:
df.head()

,time_to_collect,avg_bill_methods,debtor_volume_handled,bill_amount_collected,total_actual,total_cost,success_rate,negotiation_skills,collector_experience
0,0,sms or WA,7,14400000,35700000,277706,40.00,Baik,4
1,11,panggilan,35,79000000,160500000,22382611,49.00,Buruk,0
2,4,sms or WA,6,7100000,26700000,205389,27.00,Buruk,0
3,29,datang ke tempat,63,782000000,1395000000,482942548,56.00,Cukup baik,2
4,22,datang ke tempat,35,346000000,485500000,151035603,71.00,Cukup baik,2


In [5]:
df["avg_bill_methods"].value_counts()

avg_bill_methods
datang ke tempat    481
sms or WA           198
panggilan           162
surat panggilan     159
Name: count, dtype: int64

In [6]:
bill_methods = {"sms or WA": 0, "surat panggilan": 1, "panggilan": 2, "datang ke tempat": 3}
nego = {"Buruk": 0, "Cukup baik": 1, "Baik": 2}
columns = {"avg_bill_methods": bill_methods, "negotiation_skills": nego}
for i in df.columns:
    if df[i].dtype == "category" or df[i].dtype == "object":
        df[i] = [columns[i][j] for j in df[i]]

y = df["total_cost"]
df.drop("total_cost", axis=1, inplace=True)             
scaler = RobustScaler().fit(df[[i for i in df.columns if i not in columns]])
df2 = pd.DataFrame(scaler.transform(df[[i for i in df.columns if i not in columns]]), columns=df[[i for i in df.columns if i not in columns]].columns)
for i in columns:
    df2[i] = df[i]
df2.head(5)

,time_to_collect,debtor_volume_handled,bill_amount_collected,total_actual,success_rate,collector_experience,avg_bill_methods,negotiation_skills
0,-1.00,-1.35,-0.46,-0.52,-0.36,0.50,0,2
1,-0.27,-0.13,-0.28,-0.34,-0.16,-0.50,2,0
2,-0.73,-1.39,-0.48,-0.53,-0.66,-0.50,0,0
3,0.93,1.09,1.64,1.39,0.00,0.00,3,1
4,0.47,-0.13,0.45,0.11,0.34,0.00,3,1


In [7]:
X = df2
y_ = RobustScaler().fit(y.to_numpy().reshape(-1, 1))
y = y_.transform(y.to_numpy().reshape(-1, 1))
y = pd.Series(y.ravel())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)
reg = LazyRegressor(verbose=0, ignore_warnings=True)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
models

 17%|█▋        | 7/42 [00:00<00:01, 18.19it/s]

100%|██████████| 42/42 [00:05<00:00,  7.48it/s]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000120 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 718
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 8
[LightGBM] [Info] Start training from score 0.296629
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
AdaBoostRegressor,0.67,0.68,0.40,0.06
MLPRegressor,0.65,0.67,0.41,0.42
RandomForestRegressor,0.64,0.66,0.42,1.13
BaggingRegressor,0.64,0.65,0.42,0.15
NuSVR,0.62,0.64,0.43,0.08
SVR,0.62,0.63,0.43,0.05
GradientBoostingRegressor,0.61,0.62,0.44,0.31
ExtraTreesRegressor,0.60,0.62,0.44,0.66
OrthogonalMatchingPursuitCV,0.56,0.58,0.46,0.03


In [8]:
y.describe()

count   1000.00
mean       0.32
std        0.68
min       -0.25
25%       -0.25
50%        0.00
75%        0.75
max        2.51
dtype: float64

In [9]:
y = y_.inverse_transform(y.to_numpy().reshape(-1, 1))
y = pd.Series(y.ravel())
y.describe()

count        1000.00
mean    139633105.12
std     166600490.67
min         25263.00
25%       1928856.50
50%      62215627.00
75%     247064723.00
max     678041154.00
dtype: float64